# Udacity Machine Learning Capstone Project

## State Farm Distracted Driver Detection (Can computer vision spot distracted drivers?)

---
### The Road Ahead

We break the notebook into separate steps.  Feel free to use the links below to navigate the notebook.

* [Step 0](#step0): Import Datasets
* [Step 1](#step1): Data Analysis and Preprocessing
* [Step 2](#step2): Create a CNN to classify driver images (from scratch)
* [Step 3](#step3): Use a CNN to classify driver images (using transfer learning on pre-trained VGG16)
* [Step 4](#step4): Create a CNN to classify driver images (using transfer learning on pre-trained ResNet50)
* [Step 5](#step5): Algorithm test result

---
<a id='step0'></a>
## Step 0: Import Datasets

### Import Driver Image Dataset

In the code cell below, we import a dataset of driver images. We populate a few variables through the use of the `load_files` function from the scikit-learn library:
- `input_train_files`, `input_test_files` - numpy arrays containing file paths to images
- `input_train_targets`, `input_test_targets` - numpy arrays containing onehot-encoded classification labels 
- `label_names` - list of string-valued label codes of driver behaviors for translating labels :

    * c0: normal driving
    * c1: texting - right
    * c2: talking on the phone - right
    * c3: texting - left
    * c4: talking on the phone - left
    * c5: operating the radio
    * c6: drinking
    * c7: reaching behind
    * c8: hair and makeup
    * c9: talking to passenger

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from sklearn.cross_validation import train_test_split

# define function to load train and test image datasets provided by State Farm
def load_dataset(path):
    data = load_files(path)
    driver_files = np.array(data['filenames'])
    driver_targets = np_utils.to_categorical(np.array(data['target']), 10)
    return driver_files, driver_targets

# load original train and test datasets provided by State Farm
input_train_files, input_train_targets = load_dataset('imgs/train')
input_test_files, input_test_targets = load_dataset('imgs/test')

print('There are %s total driver images provided by State Farm:' % len(np.hstack([input_train_files, input_test_files])))
print('%d train driver images' % len(input_train_files))
print('%d test driver images\n' % len(input_test_files))

# load list of label codes of driver behaviors
label_names = [item[11:13] for item in sorted(glob("imgs/train/*/"))]

print('There are %d total driver behavior categories.' % len(label_names))

# for saving memory
del input_test_targets

Using TensorFlow backend.


There are 102150 total driver images provided by State Farm:
22424 train driver images
79726 test driver images

There are 10 total driver behavior categories.


In [2]:
# Since the number of driver images in original input train dataset provided by State Farm is too large,
# for avoidance of running out of memory when we transform them into tensors to feed and train the CNN models,
# we randomly sample a particular ratio (0.4) of them to use, and ignore the remaining portion (ratio 0.6).
ignore_ratio = 0.6
use_files, non_use_files, use_targets, non_use_targets = \
            train_test_split(input_train_files, input_train_targets, test_size=ignore_ratio, random_state=5)

print('To avoid running out of memory,')
print('we select %s images from original train set' % len(use_files))
print('and ignore the remaining %s images.\n' % len(non_use_files))

# shuffle and split the sampled "use" dataset into training set (80%) and validation set (20%)
train_files, valid_files, train_targets, valid_targets = \
            train_test_split(use_files, use_targets, test_size=0.2, random_state=5)

print('Among the %s randomly selected images,' % len(use_files))
print('we use %d images for training and' % len(train_files))
print('we use %d images for validation.' % len(valid_files))

# for saving memory
del use_files, non_use_files, use_targets, non_use_targets

To avoid running out of memory,
we select 8969 images from original train set
and ignore the remaining 13455 images.

Among the 8969 randomly selected images,
we use 7175 images for training and
we use 1794 images for validation.


In [3]:
# a list of 79726 test image filenames
test_image_filename_list = [test_file_path[15:] for test_file_path in input_test_files]

---
<a id='step1'></a>
## Step 1: Data Analysis and Preprocessing

### Data Analysis

In the code cells below, we read the **driver_imgs_list.csv** file provided by State Farm. This csv file is a list of original input training images, their `subject` (driver id), and `classname` (label id). We then analyze this original input train data set. There is just a little bit size imbalance between different classes, i.e., roughly balance. Note that class 'c0' has the maximum number of 2489 images and class 'c8' has the minimum number of 1911 images among all the 10 classes.

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
from IPython.display import display

training_images_df = pd.read_csv("driver_imgs_list.csv")
# display the top 5 data items in "driver_imgs_list.csv"
display(training_images_df.head())

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [5]:
# display information of original input train images
display(training_images_df.describe())

,subject,classname,img
count,22424,22424,22424
unique,26,10,22424
top,p021,c0,img_4530.jpg
freq,1237,2489,1


In [6]:
# show the 10 classes of driver images and the number of images in each class in the original input train data set
print(training_images_df['classname'].value_counts(sort=True))

c0    2489
c3    2346
c4    2326
c6    2325
c2    2317
c5    2312
c1    2267
c9    2129
c7    2002
c8    1911
Name: classname, dtype: int64


### Data Preprocessing

When using TensorFlow as backend, Keras CNNs require a 4D array (which we'll also refer to as a 4D tensor) as input, with shape

$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$

where `nb_samples` corresponds to the total number of images (or samples), and `rows`, `columns`, and `channels` correspond to the number of rows, columns, and channels for each image, respectively.  

The `path_to_tensor` function below takes a string-valued file path to a color image as input and returns a 4D tensor suitable for supplying to a Keras CNN.  The function first loads the image and resizes it to a square image that is $224 \times 224$ pixels.  Next, the image is converted to an array, which is then resized to a 4D tensor.  In this case, since we are working with color images, each image has three channels.  Likewise, since we are processing a single image (or sample), the returned tensor will always have shape

$$
(1, 224, 224, 3).
$$

The `paths_to_tensor` function takes a numpy array of string-valued image paths as input and returns a 4D tensor with shape 

$$
(\text{nb_samples}, 224, 224, 3).
$$

Here, `nb_samples` is the number of samples, or number of images, in the supplied array of image paths.  It is best to think of `nb_samples` as the number of 3D tensors (where each 3D tensor corresponds to a different image) in the dataset!

In [7]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In the code cell below, we rescale the images by dividing every pixel in every image by 255.

In [8]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255

# the 79726 test images will lead to run out of memory
#test_tensors = paths_to_tensor(input_test_files).astype('float32')/255

100%|█████████████████████████████████████████████████████████████████████████████| 1794/1794 [00:13<00:00, 129.98it/s]


---
<a id='step2'></a>
## Step 2: Create a CNN to classify driver images (from scratch)

We will use Keras and Tensorflow to implement our first CNN model. In this step, we will
provide the architecture of the first CNN model we design. And then test the performance
result of this CNN model.

### Model Architecture

We create a CNN to classify driver behaviors. At the end of code cell block, we summarize the layers of the CNN model by executing the line:
    
        model.summary()

We use three convolotion layers followed by three max pooling layers interleavingly and then use two fully connected layers behind in the CNN architecture. We also adopt batch_normalization layers between each convolution layer or dense layer and their activation layer to avoid covariate shift and to accelerate the training process. The number of filters in each convolution layer is twice to the previous one (this is a common practice), and we choose 16, 32, and 64 filters to extract the feature maps (regional information). The window size of feature filter in each convolution layer and also the pool size in each max pooling layer are both (2,2), and it's also a kind of typical choices. We set the padding parameter to be 'same' for not lost information near matrix boundaries. The activation function in each layer beside output is ReLU for dealing with the vanishing gradient problem, and that in output layer is SoftMax for calculation of probabilities on the multi-classes. In max pooling layers, we set the strides parameter to be 2 for half both length and width of each 2D feature map (dimensional reduction), and such strides setting is also typical. Before fully connected layers, we use the GlobalAveragePooling2D layer, which can immediately reduce the amount of parameters and avoid overfitting as well as save much time. We adopt the dropout layers with probability 0.2 to reduce opportunity of overfitting. We choose the number of nodes to be 64 in the first fully connected layer for an initial try, and due to the 10 classes of driver behaviors, the number of nodes in output layer is 10.

In [13]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding='same', input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2, strides=2, padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(GlobalAveragePooling2D())

model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(10))
model.add(BatchNormalization())
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 16)      64        
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 56, 56, 32)        128       
__________

### Compile the Model

We choose RMSprop optimizer (RMS: root mean squared error). It decreases the learning rate by dividing it by an exponentially decaying average of squared gradients. We use categorical cross entropy to be the loss error function.

In [17]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the Model

We train the first CNN model we design in the code cell below. We adopt `epochs` parameter to be 10 and `batch_size` parameter to be 20 for an initial try. Use model checkpointing to save the model that attains the best validation loss.

In [18]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=10, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 7175 samples, validate on 1794 samples
Epoch 1/10
7175/7175 [==============================] - 1042s - loss: 2.2047 - acc: 0.1994 - val_loss: 3.0725 - val_acc: 0.1165
Epoch 2/10
7175/7175 [==============================] - 1037s - loss: 1.8863 - acc: 0.3192 - val_loss: 3.1912 - val_acc: 0.1600
Epoch 3/10
7175/7175 [==============================] - 1041s - loss: 1.7287 - acc: 0.3851 - val_loss: 2.5879 - val_acc: 0.2469
Epoch 4/10
7175/7175 [==============================] - 1047s - loss: 1.6151 - acc: 0.4283 - val_loss: 2.1443 - val_acc: 0.2759
Epoch 5/10
7175/7175 [==============================] - 1076s - loss: 1.5264 - acc: 0.4714 - val_loss: 2.8780 - val_acc: 0.2336
Epoch 6/10
7175/7175 [==============================] - 1001s - loss: 1.4474 - acc: 0.4994 - val_loss: 2.8416 - val_acc: 0.2464
Epoch 7/10
7175/7175 [==============================] - 1015s - loss: 1.3518 - acc: 0.5339 - val_loss: 1.7570 - val_acc: 0.3685
Epoch 8/10
7175/7175 [==============================] - 

### Load the Model with the Best Validation Loss

In [19]:
model.load_weights('saved_models/weights.best.from_scratch.hdf5')

### Test the Model

In the code cell below, we test our first CNN model on the testing data set of driver images. The prediction probability results of all the 79726 test images are written into the csv file: **CNN_1_test_probability.csv**, following the submission format defined by Kaggle:

- img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
- img_0.jpg,1,0,0,0,0,...,0
- img_1.jpg,0.3,0.1,0.6,0,...,0
- ...

**The score (evaluation metrics: logarithmic loss function) of our first CNN model is 2.60072.**

**The test result of our first CNN model is ranked 1355 out of 1440 in public leader board, and ranked 1340 out of 1440 in private leader board.**

In [9]:
# for saving memory
del train_tensors, valid_tensors

In [20]:
driver_behavior_predictions = []
for test_file in tqdm(input_test_files):
    test_tensor = path_to_tensor(test_file)
    test_tensor = np.vstack(test_tensor).astype('float32')/255
    driver_behavior_predictions.append(model.predict(np.expand_dims(test_tensor, axis=0))[0])

#driver_behavior_predictions = [model.predict(np.expand_dims(test_tensor, axis=0))[0] for test_tensor in test_tensors]

test_image_probability_csv = np.column_stack((np.asarray(test_image_filename_list), \
                                              np.asarray(driver_behavior_predictions, dtype=np.float32)))

np.savetxt('submission/CNN_1_test_probability.csv', test_image_probability_csv, delimiter=',', \
           comments='', newline='\n', fmt='%s', header='img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9')

# get index of predicted dog breed for each image in test set
#dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
#test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
#print('Test accuracy: %.4f%%' % test_accuracy)

100%|██████████████████████████████████████████████████████████████████████████| 79726/79726 [1:16:49<00:00, 17.23it/s]


In [ ]:
# for saving memory
del driver_behavior_predictions, test_image_probability_csv

---
<a id='step3'></a>
## Step 3: Use a CNN to classify driver images (using transfer learning on pre-trained VGG16)

To reduce training time without sacrificing accuracy, we will train a CNN model using
transfer learning. In this step, our CNN model will use the pre-trained VGG16 model as a
**fixed feature extractor**, where the last convolutional output of VGG16 is fed as input to our
model.

## VGG16
### Import VGG16 model

In [8]:
from keras.applications.vgg16 import VGG16

# https://keras.io/applications/#vgg16
# NOT include the 3 fully-connected layers at the top of the network
model = VGG16(include_top=False)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

### Obtain Bottleneck Features

In [23]:
bottleneck_features_valid = []

for img_path in tqdm(valid_files):
    tensor = path_to_tensor(img_path)
    tensor = np.vstack(tensor).astype('float32')/255
    bottleneck_features_valid.append(model.predict(np.expand_dims(tensor, axis=0))[0])

bottleneck_features_valid = np.asarray(bottleneck_features_valid, dtype=np.float32)

#bottleneck_features_valid = \
#        np.asarray([model.predict(np.expand_dims(valid_tensor, axis=0))[0] for valid_tensor in valid_tensors], dtype=np.float32)

np.save(open('bottleneck_features/driver_VGG16_valid.npy', 'wb'), bottleneck_features_valid)

100%|██████████████████████████████████████████████████████████████████████████████| 1794/1794 [50:22<00:00,  1.84s/it]


In [27]:
# for saving memory
del bottleneck_features_valid

In [28]:
bottleneck_features_train = []

for img_path in tqdm(train_files):
    tensor = path_to_tensor(img_path)
    tensor = np.vstack(tensor).astype('float32')/255
    bottleneck_features_train.append(model.predict(np.expand_dims(tensor, axis=0))[0])

bottleneck_features_train = np.asarray(bottleneck_features_train, dtype=np.float32)

#bottleneck_features_train = \
#        np.asarray([model.predict(np.expand_dims(train_tensor, axis=0))[0] for train_tensor in train_tensors], dtype=np.float32)

np.save(open('bottleneck_features/driver_VGG16_train.npy', 'wb'), bottleneck_features_train)

100%|████████████████████████████████████████████████████████████████████████████| 7175/7175 [3:35:43<00:00,  1.81s/it]


In [28]:
# for saving memory
del bottleneck_features_train

In [9]:
bottleneck_features_valid = np.load('bottleneck_features/driver_VGG16_valid.npy')
bottleneck_features_train = np.load('bottleneck_features/driver_VGG16_train.npy')

### Model Architecture

In our second CNN model using transfer learning of VGG16, we add a global average pooling (GAP) layer and two fully connected (dense) layers, where the latter dense layer contains 10 nodes for each of the 10 driver behavior categories, and the output layer is equipped with a softmax activation function to calculate the probabilities for multi-classes (10 classes).

In [10]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

VGG16_model = Sequential()

VGG16_model.add(GlobalAveragePooling2D(input_shape=bottleneck_features_train.shape[1:]))

VGG16_model.add(Dense(64))
VGG16_model.add(BatchNormalization())
VGG16_model.add(Activation('relu'))
VGG16_model.add(Dropout(0.2))

VGG16_model.add(Dense(10))
VGG16_model.add(BatchNormalization())
VGG16_model.add(Activation('softmax'))

VGG16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32832     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
_________________________________________________________________
batch_normalization_2 (Batch (None, 10)                40        
__________

### Compile the Model

We choose RMSprop optimizer (RMS: root mean squared error). It decreases the learning rate by dividing it by an exponentially decaying average of squared gradients. We use categorical cross entropy to be the loss error function.

In [11]:
VGG16_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### Train the Model

We adopt `epochs` parameter to be 30 to reduce the opportunity of overfitting and `batch_size` parameter to be 20 to accelerate the gradient descent iterations. We also use model checkpointing to save the model that attains the best validation loss.

In [12]:
from keras.callbacks import ModelCheckpoint  

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.hdf5', 
                               verbose=1, save_best_only=True)

VGG16_model.fit(bottleneck_features_train, train_targets, 
                validation_data=(bottleneck_features_valid, valid_targets),
                epochs=30, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 7175 samples, validate on 1794 samples
Epoch 1/30
7175/7175 [==============================] - 4s - loss: 1.4040 - acc: 0.5884 - val_loss: 1.5957 - val_acc: 0.4554
Epoch 2/30
7175/7175 [==============================] - 3s - loss: 0.8134 - acc: 0.8226 - val_loss: 1.3341 - val_acc: 0.5312
Epoch 3/30
7175/7175 [==============================] - 3s - loss: 0.6101 - acc: 0.8718 - val_loss: 0.8198 - val_acc: 0.7441
Epoch 4/30
7175/7175 [==============================] - 3s - loss: 0.5155 - acc: 0.8888 - val_loss: 0.5323 - val_acc: 0.8562
Epoch 5/30
7175/7175 [==============================] - 3s - loss: 0.4552 - acc: 0.8948 - val_loss: 0.4259 - val_acc: 0.8963
Epoch 6/30
7175/7175 [==============================] - 3s - loss: 0.4066 - acc: 0.9044 - val_loss: 0.5458 - val_acc: 0.8333
Epoch 7/30
7175/7175 [==============================] - 3s - loss: 0.3703 - acc: 0.9090 - val_loss: 0.7237 - val_acc: 0.7659
Epoch 8/30
7175/7175 [==============================] - 3s - loss: 0.3328 - a

### Load the Model with the Best Validation Loss

In [13]:
VGG16_model.load_weights('saved_models/weights.best.VGG16.hdf5')

### Test the Model

In the code cell below, we test our second CNN model (using transfer learning of VGG16) on the testing data set of driver images. The prediction probability results of all the 79726 test images are written into the csv file: **CNN_VGG16_test_probability.csv**, following the submission format defined by Kaggle:

- img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
- img_0.jpg,1,0,0,0,0,...,0
- img_1.jpg,0.3,0.1,0.6,0,...,0
- ...

**The score (evaluation metrics: logarithmic loss function) of our second CNN model (using transfer learning of VGG16) is 1.72885.**

**The test result of our second CNN model (using transfer learning of VGG16) is ranked 1004 out of 1440 in public leader board, and ranked 746 out of 1440 in private leader board.**

In [14]:
# for saving memory
del bottleneck_features_train, bottleneck_features_valid

In [15]:
driver_behavior_predictions = []
for test_file in tqdm(input_test_files):
    test_tensor = path_to_tensor(test_file)
    test_tensor = np.vstack(test_tensor).astype('float32')/255
    test_bottleneck_feature = model.predict(np.expand_dims(test_tensor, axis=0))[0]
    driver_behavior_predictions.append(VGG16_model.predict(np.expand_dims(test_bottleneck_feature, axis=0))[0])

test_image_probability_csv = np.column_stack((np.asarray(test_image_filename_list), \
                                              np.asarray(driver_behavior_predictions, dtype=np.float32)))

np.savetxt('submission/CNN_VGG16_test_probability.csv', test_image_probability_csv, delimiter=',', \
           comments='', newline='\n', fmt='%s', header='img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9')

# get index of predicted dog breed for each image in test set
#VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

# report test accuracy
#test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
#print('Test accuracy: %.4f%%' % test_accuracy)

100%|█████████████████████████████████████████████████████████████████████████| 79726/79726 [39:08:51<00:00,  1.80s/it]


In [16]:
# for saving memory
del driver_behavior_predictions, test_image_probability_csv

In [ ]:
#from extract_bottleneck_features import *

#def VGG16_predict_breed(img_path):
#    # extract bottleneck features
#    bottleneck_feature = extract_VGG16(path_to_tensor(img_path))
#    # obtain predicted vector
#    predicted_vector = VGG16_model.predict(bottleneck_feature)
#    # return dog breed that is predicted by the model
#    return dog_names[np.argmax(predicted_vector)]

---
<a id='step4'></a>
## Step 4: Create a CNN to classify driver images (using transfer learning on pre-trained ResNet50)

In this step, instead of VGG16, we may choose to try another pre-trained model, like VGG19, Resnet50, InceptionV3, or Xception, for different pre-trained model selection of transfer learning. We can compare
the chosen pre-trained CNN model with the above one (VGG16) and check the difference and result between their prediction scores.

Here we choose ResNet50 to be the pre-trained CNN model of transfer learning below. Note that the remaining parts like number of epochs, optimizer, loss function, fully-connected layers (, etc) are not changed for comparisons of results.

## ResNet50
### Import ResNet50 model

In [8]:
from keras.applications.resnet50 import ResNet50

# https://keras.io/applications/#resnet50
# NOT include the fully-connected layer at the top of the network.
model = ResNet50(include_top=False)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, None, None, 64 9472        input_1[0][0]                    
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, None, None, 64 256         conv1[0][0]                      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 64 0           bn_conv1[0][0]                   
___________________________________________________________________________________________

### Obtain Bottleneck Features

In [18]:
bottleneck_features_valid = []

for img_path in tqdm(valid_files):
    tensor = path_to_tensor(img_path)
    tensor = np.vstack(tensor).astype('float32')/255
    bottleneck_features_valid.append(model.predict(np.expand_dims(tensor, axis=0))[0])

bottleneck_features_valid = np.asarray(bottleneck_features_valid, dtype=np.float32)

#bottleneck_features_valid = \
#        np.asarray([model.predict(np.expand_dims(valid_tensor, axis=0))[0] for valid_tensor in valid_tensors], dtype=np.float32)

np.save(open('bottleneck_features/driver_ResNet50_valid.npy', 'wb'), bottleneck_features_valid)

100%|██████████████████████████████████████████████████████████████████████████████| 1794/1794 [35:07<00:00,  1.19s/it]


In [19]:
# for saving memory
del bottleneck_features_valid

In [20]:
bottleneck_features_train = []

for img_path in tqdm(train_files):
    tensor = path_to_tensor(img_path)
    tensor = np.vstack(tensor).astype('float32')/255
    bottleneck_features_train.append(model.predict(np.expand_dims(tensor, axis=0))[0])

bottleneck_features_train = np.asarray(bottleneck_features_train, dtype=np.float32)

#bottleneck_features_train = \
#        np.asarray([model.predict(np.expand_dims(train_tensor, axis=0))[0] for train_tensor in train_tensors], dtype=np.float32)

np.save(open('bottleneck_features/driver_ResNet50_train.npy', 'wb'), bottleneck_features_train)

100%|████████████████████████████████████████████████████████████████████████████| 7175/7175 [2:17:35<00:00,  1.13s/it]


In [21]:
# for saving memory
del bottleneck_features_train

In [9]:
bottleneck_features_valid = np.load('bottleneck_features/driver_ResNet50_valid.npy')
bottleneck_features_train = np.load('bottleneck_features/driver_ResNet50_train.npy')

### Model Architecture

In [10]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

ResNet50_model = Sequential()

ResNet50_model.add(GlobalAveragePooling2D(input_shape=bottleneck_features_train.shape[1:]))

ResNet50_model.add(Dense(64))
ResNet50_model.add(BatchNormalization())
ResNet50_model.add(Activation('relu'))
ResNet50_model.add(Dropout(0.2))

ResNet50_model.add(Dense(10))
ResNet50_model.add(BatchNormalization())
ResNet50_model.add(Activation('softmax'))

ResNet50_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                131136    
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
activation_50 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
_________________________________________________________________
batch_normalization_2 (Batch (None, 10)                40        
__________

### Compile the Model

We choose RMSprop optimizer (RMS: root mean squared error). It decreases the learning rate by dividing it by an exponentially decaying average of squared gradients. We use categorical cross entropy to be the loss error function.

In [11]:
ResNet50_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

### Train the Model

We adopt `epochs` parameter to be 30 to reduce the opportunity of overfitting and `batch_size` parameter to be 20 to accelerate the gradient descent iterations. We also use model checkpointing to save the model that attains the best validation loss.

In [12]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.ResNet50.hdf5', 
                               verbose=1, save_best_only=True)

ResNet50_model.fit(bottleneck_features_train, train_targets, 
                   validation_data=(bottleneck_features_valid, valid_targets),
                   epochs=30, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 7175 samples, validate on 1794 samples
Epoch 1/30
7175/7175 [==============================] - 5s - loss: 1.6014 - acc: 0.4893 - val_loss: 2.7172 - val_acc: 0.2469
Epoch 2/30
7175/7175 [==============================] - 4s - loss: 1.0840 - acc: 0.7102 - val_loss: 5.1464 - val_acc: 0.2012
Epoch 3/30
7175/7175 [==============================] - 4s - loss: 0.8839 - acc: 0.7682 - val_loss: 4.6469 - val_acc: 0.2419
Epoch 4/30
7175/7175 [==============================] - 4s - loss: 0.7427 - acc: 0.7994 - val_loss: 1.7209 - val_acc: 0.4387
Epoch 5/30
7175/7175 [==============================] - 4s - loss: 0.6576 - acc: 0.8234 - val_loss: 6.3729 - val_acc: 0.2090
Epoch 6/30
7175/7175 [==============================] - 4s - loss: 0.6027 - acc: 0.8329 - val_loss: 4.6145 - val_acc: 0.3322
Epoch 7/30
7175/7175 [==============================] - 4s - loss: 0.5454 - acc: 0.8539 - val_loss: 2.2455 - val_acc: 0.3846
Epoch 8/30
7175/7175 [==============================] - 3s - loss: 0.4929 - a

### Load the Model with the Best Validation Loss

In [13]:
ResNet50_model.load_weights('saved_models/weights.best.ResNet50.hdf5')

### Test the Model

In the code cell below, we test our CNN model using transfer learning of ResNet50 on the testing data set of driver images. The prediction probability results of all the 79726 test images are written into the csv file: **CNN_ResNet50_test_probability.csv**, following the submission format defined by Kaggle:

- img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
- img_0.jpg,1,0,0,0,0,...,0
- img_1.jpg,0.3,0.1,0.6,0,...,0
- ...

**The score (evaluation metrics: logarithmic loss function) of our third CNN model (using transfer learning of ResNet50) is 3.08525.**

**The test result of our third CNN model (using transfer learning of ResNet50) is ranked 1380 out of 1440 in public leader board, and ranked 1380 out of 1440 in private leader board.**

In [14]:
# for saving memory
del bottleneck_features_train, bottleneck_features_valid

In [ ]:
driver_behavior_predictions = []
for test_file in tqdm(input_test_files):
    test_tensor = path_to_tensor(test_file)
    test_tensor = np.vstack(test_tensor).astype('float32')/255
    test_bottleneck_feature = model.predict(np.expand_dims(test_tensor, axis=0))[0]
    driver_behavior_predictions.append(ResNet50_model.predict(np.expand_dims(test_bottleneck_feature, axis=0))[0])

test_image_probability_csv = np.column_stack((np.asarray(test_image_filename_list), \
                                              np.asarray(driver_behavior_predictions, dtype=np.float32)))

np.savetxt('submission/CNN_ResNet50_test_probability.csv', test_image_probability_csv, delimiter=',', \
           comments='', newline='\n', fmt='%s', header='img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9')

# get index of predicted dog breed for each image in test set
#VGG16_predictions = [np.argmax(VGG16_model.predict(np.expand_dims(feature, axis=0))) for feature in test_VGG16]

# report test accuracy
#test_accuracy = 100*np.sum(np.array(VGG16_predictions)==np.argmax(test_targets, axis=1))/len(VGG16_predictions)
#print('Test accuracy: %.4f%%' % test_accuracy)

100%|█████████████████████████████████████████████████████████████████████████| 79726/79726 [25:44:08<00:00,  1.18s/it]


In [ ]:
# for saving memory
del driver_behavior_predictions, test_image_probability_csv

In [15]:
#def Resnet50_predict_breed(img_path):
#    # extract bottleneck features
#    # VGG19, Resnet50, InceptionV3, or Xception
#    bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
#    # obtain predicted vector
#    predicted_vector = Resnet50_model.predict(bottleneck_feature)
#    # return dog breed that is predicted by the model
#    return dog_names[np.argmax(predicted_vector)]

---
<a id='step5'></a>
## Step 5: Algorithm test result

We choose the best test result (the lowest score of the evaluation metric of log-loss error function) among the CNN models we
construct above to be the final choice of our proposed algorithm.

**The CNN model we construct above with the best (lowest) test log-loss score is: transfer learning on VGG16, and it has the test score of 1.72885.**